In [2]:
import sys

root = ".."
sys.path.append(root)

In [5]:
import yaml
import os


In [ ]:
path:
  info_path: /data2/backup/maedera-workspace/TGGATE/rep/projects/2104projects/info/210403info.csv
  ft_list_path:
  fold_path: /data2/backup/maedera-workspace/TGGATE/rep/projects/2104projects/experiments/210403effnetb4-baseline/fold.csv
  trained_path: 
  tile_dir: /data0/TGGATE/tiles
  save_path: ../experiments/210807effnetb4-baseline-singleft
log:
  logger_name: 210807log.log
random:
  random_state: 123
CV:  
  create_fold: False
  n_splits: 5
  split_type: GroupKFold
  metric_list:
    - macro_auroc
    - macro_accuracy
  fold_list:
    - 0
model:
  model_name: tf_efficientnet_b4_ns
  feature_extraction: False
  pretrained: True
  load_trained: False
preprocess:
  image_size: 512
  transform_list:
    - HorizontalFlip
    - VerticalFlip
  n_tiles: 100
process:
  num_workers: 4
  train_batch_size: 32
  valid_batch_size: 32
training:
  criterion_name: BCE
  optimizer_name: Adam
  learning_rate: 0.0005
  n_epochs: 20
  verbose: 20
device:
  device: cuda
debug: False

In [7]:
from clmodel.train import train_loop
from clmodel.utils import Logger
from clmodel.dataset import load_features
import pandas as pd


In [8]:
info_path = "../data/TGGATEs/processed/info.csv"
ft_list_path = "../data/TGGATEs/processed/ft_list.txt"

info_df = pd.read_csv(info_path)
ft_list = load_features(ft_list_path)

info_df["GROUP"] = 100 * info_df["EXP_ID"] + info_df["GROUP_ID"]

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
import torch
from clmodel.utils import fix_seed

torch.backends.cudnn.benchmark=True
random_state = 123
fix_seed(random_state)


In [ ]:
logger = Logger(name = logger_name, save_dir = save_path)

fold = Fold(info_df, "INDV_ID", ["FILE"], ft_list, group_col = "GROUP",
        random_state = random_state, split_type = split_type, n_splits = n_splits,
        save_path = os.path.join(save_path, "fold.csv"))

metrics = []
for mtr_name in metric_list:
    if mtr_name == "macro_auroc":
        mtr = Metric("macro AUROC", macro_auroc, "+", limit = (0, 1))
        metrics.append(mtr)
    elif mtr_name == "auroc":
        mtr = Metric("AUROC", auroc, "multilabel", limit = (0, 1))
        metrics.append(mtr)
    elif mtr_name == "macro_accuracy":
        mtr = Metric("macro acc", macro_accuracy, "+", limit = (0, 1))
        metrics.append(mtr)
    elif mtr_name == "accuracy":
        mtr = Metric("acc", accuracy, "multilabel", limit = (0, 1))
        metrics.append(mtr)
    elif mtr_name == "macro_balanced_accuracy":
        mtr = Metric("macro balanced acc", macro_balanced_accuracy, "+", limit = (0, 1))
        metrics.append(mtr)
    elif mtr_name == "macro_r2_score":
        mtr = Metric("macro r2 score", macro_r2_score, "+", limit = (0, 1))
        metrics.append(mtr)

CV = CrossValidation(fold, metrics = metrics)

model = Model(model_name, len(ft_list),
    feature_extraction=feature_extraction, pretrained=pretrained)
if load_trained:
    model.load_state_dict(torch.load(trained_path))
print("created the model.")
train_transform = [albu.RandomCrop(image_size, image_size)]

for transform in transform_list:
    if transform == "HorizontalFlip":
        train_transform.append(albu.HorizontalFlip(p=0.5))
    elif transform == "VerticalFlip":
        train_transform.append(albu.VerticalFlip(p=0.5))

train_transform.append(albu.Normalize(
                        mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225],
                    ))

train_transform.append(ToTensorV2())

train_transform = albu.Compose(train_transform)

valid_transform = albu.Compose([albu.RandomCrop(image_size, image_size),
                    albu.Normalize(
                        mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225],
                    ),
                    ToTensorV2(),])

if tile_dict_flag:
    from glob import glob
    tile_dict = {}
    for td in tile_dir:
        svss = [f.split("/")[-1] for f in glob(td+"/*.svs") if len(f.split("."))==2]
        for s in svss:
            tile_dict[s] = td

for fold_i in fold_list:
    train_info, valid_info = CV[fold_i]
    if tile_dict_flag:
        train_dataset= TGGATEDataset(train_info, mode, tile_dir=None, n_tiles=n_tiles, tile_dict=tile_dict,
            ft_list = ft_list, compound_list = None, pp_list = None, 
            transform = train_transform, debug = debug, he_aug=False)
            
        valid_dataset= TGGATEDataset(valid_info, mode, tile_dir=None, n_tiles=n_tiles, tile_dict=tile_dict,
            ft_list = ft_list, compound_list = None, pp_list = None, 
            transform = valid_transform, debug = debug, he_aug=False)
    else:
        print(tile_dict_flag)
        #with open("../processed_data/210417vein.pkl", "rb") as f:
        #    st = pickle.load(f)
        train_dataset= TGGATEDataset(train_info, mode, tile_dir, n_tiles,
            ft_list = ft_list, compound_list = None, pp_list = None, 
            transform = train_transform, debug = debug, he_aug=False, normalize=False)
            
        valid_dataset= TGGATEDataset(valid_info, mode, tile_dir, n_tiles,
            ft_list = ft_list, compound_list = None, pp_list = None, 
            transform = valid_transform, debug = debug, normalize=False)

    train_loader = DataLoader(dataset=train_dataset, num_workers=num_workers,
        batch_size=train_batch_size,
        drop_last=True,
        #shuffle=True)
        sampler=TGGATEBalancedSampler(train_dataset, 0.1))
    valid_loader = DataLoader(dataset=valid_dataset, num_workers=num_workers,
        batch_size=valid_batch_size, shuffle=False)

    if criterion_name == "BCE":
        criterion = nn.BCEWithLogitsLoss()
        preprocess = lambda x: x.sigmoid()
    elif criterion_name == "WeightedBCE":
        positive_weight = train_info[ft_list].mean().values.astype(np.float32)
        positive_weight = torch.tensor((1 - positive_weight)/(positive_weight + 1e-5))
        criterion = WeightedBCELossWithLogits(positive_weight=positive_weight, negative_weight=torch.tensor(1), device=device)
        preprocess = lambda x: x.sigmoid()
    elif criterion_name == "FocalBCE":
        criterion = FocalBCELossWithLogits(gamma=1)
        preprocess = lambda x: x.sigmoid()
    elif criterion_name == "MSE":
        criterion = nn.MSELoss()
        preprocess = lambda x: x
    if optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = CosineAnnealingLR(optimizer, n_epochs, 5e-7)
    model = model.to(device)
    train_history, valid_history, best_score, best_predictions = \
        train_loop(model, train_loader, valid_loader, fold_i, criterion,
        optimizer, device, n_epochs, scheduler, metrics, save_path,
        model_name, preprocess, verbose, logger)
    print(best_score)
    visualize(train_history, valid_history, metrics, save_dir=save_path)
    for mtr_name, predictions in best_predictions.items():
        CV.set_oof_pred(fold_i, predictions[0], mtr_name)
    CV.save_oof(os.path.join(save_path, "oof_df.csv"))

    with open(os.path.join(save_path, f"train_result_fold{fold_i}.pickle"), "wb") as f:
        pickle.dump((train_history, valid_history, best_score, best_predictions), f)
